In [1]:
import os
from dotenv import load_dotenv 
load_dotenv()

True

In [2]:
from langchain_huggingface import HuggingFaceEmbeddings
hf_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

C:\Users\muham\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
len(hf_embeddings.embed_query("hello ai"))

384

<h3>Step 1 : Initialize pinecone API key </h3>

In [4]:
from pinecone import Pinecone
import os
pinecone_api_key = os.getenv("PINECONE_API_KEY")

In [5]:
pc = Pinecone(api_key=pinecone_api_key)
pc

In [6]:
from pinecone import ServerlessSpec

<h3>Step 2 : Create pinecone Index</h3>

In [ ]:
index_name = "agentic-agent-class"
pc.has_index(index_name)

True

In [ ]:


if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension = 384,
        metric = "cosine",
        spec = ServerlessSpec(cloud='aws', region='us-east-1'),
    )

<h3>Step 3 : Load that Index </h3>

In [10]:
index = pc.Index(index_name)

<h3>Step 4 : Setting up Vector Store</h3>

You're telling LangChain:

“Hey, use this Pinecone index for storing and retrieving vectorized documents, and use this embedding model to convert text into vectors.”

In [16]:
#Creating Vector Store
from langchain_pinecone import PineconeVectorStore
pc_vectorstore = PineconeVectorStore(index=index,embedding=hf_embeddings)

<h3>Step 5 : Add Documents</h3>

<h5> 5.1 : documents </h5>

In [19]:
from langchain_core.documents import Document
document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},#additional info
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

<h5> 5.2 : Generating keys for each document</h5>

In [24]:

from uuid import uuid4
uuids = [ str(uuid4()) for _ in range(len(documents)) ]
uuids

['080d696f-bd9d-4b77-9369-a28d48ae0609',
 '10f54f6e-ee16-4188-b515-7ba1e7d167d6',
 '4518a287-3c8a-4db8-bb61-a8f6368be835',
 'af142375-c9ce-43fe-887f-1260368b83dc',
 '5e1e11ee-c934-45df-88f3-82b1fbcfd427',
 '84de0c8a-1f95-4e76-ae4e-8a7f90ec5491',
 '1b6686d0-5575-4d9a-935e-c9a75cba73e3',
 '886cf346-0423-4672-b5a3-0195b65b6a11',
 '0605e264-5a9f-43cf-8a8d-ee9d5d87b85f',
 '7595dc34-ee3a-47e0-86f8-50a914e0d8a5']

<h5> Step 6 : Adding documents and keys to Vector Store</h5>

In [40]:
pc_vectorstore.add_documents(documents=documents, ids=uuids)

['080d696f-bd9d-4b77-9369-a28d48ae0609',
 '10f54f6e-ee16-4188-b515-7ba1e7d167d6',
 '4518a287-3c8a-4db8-bb61-a8f6368be835',
 'af142375-c9ce-43fe-887f-1260368b83dc',
 '5e1e11ee-c934-45df-88f3-82b1fbcfd427',
 '84de0c8a-1f95-4e76-ae4e-8a7f90ec5491',
 '1b6686d0-5575-4d9a-935e-c9a75cba73e3',
 '886cf346-0423-4672-b5a3-0195b65b6a11',
 '0605e264-5a9f-43cf-8a8d-ee9d5d87b85f',
 '7595dc34-ee3a-47e0-86f8-50a914e0d8a5']

In [42]:
index.describe_index_stats()

{'dimension': 384,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 10}},
 'total_vector_count': 10,
 'vector_type': 'dense'}

In [71]:
#Checking  Similarity
pc_vectorstore.similarity_search("do you robbed",k=3)

[Document(id='af142375-c9ce-43fe-887f-1260368b83dc', metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.'),
 Document(id='080d696f-bd9d-4b77-9369-a28d48ae0609', metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.'),
 Document(id='84de0c8a-1f95-4e76-ae4e-8a7f90ec5491', metadata={'source': 'website'}, page_content='Is the new iPhone worth the price? Read this review to find out.')]

In [47]:
#Checking  Similarity
pc_vectorstore.similarity_search("do you robbed",filter={"source":"tweet"},)

[Document(id='080d696f-bd9d-4b77-9369-a28d48ae0609', metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.'),
 Document(id='7595dc34-ee3a-47e0-86f8-50a914e0d8a5', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :('),
 Document(id='5e1e11ee-c934-45df-88f3-82b1fbcfd427', metadata={'source': 'tweet'}, page_content="Wow! That was an amazing movie. I can't wait to see it again."),
 Document(id='886cf346-0423-4672-b5a3-0195b65b6a11', metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic applications!')]

<h5>Step 6 : Configure Retriever</h5>

In [76]:
# Setting up the vector store
retriever = pc_vectorstore.as_retriever(
)

In [77]:
retriever.invoke("is robbery crime?")

[Document(id='af142375-c9ce-43fe-887f-1260368b83dc', metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.'),
 Document(id='84de0c8a-1f95-4e76-ae4e-8a7f90ec5491', metadata={'source': 'website'}, page_content='Is the new iPhone worth the price? Read this review to find out.'),
 Document(id='080d696f-bd9d-4b77-9369-a28d48ae0609', metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.'),
 Document(id='0605e264-5a9f-43cf-8a8d-ee9d5d87b85f', metadata={'source': 'news'}, page_content='The stock market is down 500 points today due to fears of a recession.')]

<h4>Step 7 : setup Model,Prompt,Parser</h4>

<h5>7.1 : Model</h5>

In [51]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model = "gemini-1.5-flash") 

<h5>7.2 : Prompt</h5>

In [55]:
from langchain import hub
prompt = hub.pull("rlm/rag-prompt")
print(prompt)

input_variables=['context', 'question'] input_types={} partial_variables={} metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})]


In [ ]:
#just skipped because of low time
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate(
    template="",
    input_variables=["question","context"]
)

<h5>7.3 : Parser</h5>

In [ ]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

<h5> Step 8 : Setup Context</h5>

In [95]:
from langchain_core.runnables import RunnablePassthrough
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)
context = {"context":retriever | format_docs, "question":RunnablePassthrough()}

<h4>Step 9 : Create RAG Chain</h4>

In [101]:
rag_chain = (
    context
    | prompt 
    | model 
    | StrOutputParser()
    )

<h5>Step 10 : Ready to query</h5>

In [102]:
rag_chain.invoke("what is llama model?")

"I don't know.  The provided text does not contain information about Llama models."

In [90]:
print(rag_chain.input_schema)


<class 'langchain_core.runnables.base.RunnableParallel<context,question>Input'>


In [80]:
model.invoke("hi")

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run--d6e21dd7-c2ba-4b86-b8e3-2222cd5df2e5-0', usage_metadata={'input_tokens': 1, 'output_tokens': 11, 'total_tokens': 12, 'input_token_details': {'cache_read': 0}})

In [86]:
prompt.invoke({
    "question":"hey",
    "context":""
    })

ChatPromptValue(messages=[HumanMessage(content="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: hey \nContext:  \nAnswer:", additional_kwargs={}, response_metadata={})])